In [2]:
import cv2
import pandas
import os
import glob
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix

In [ ]:
print(os.listdir("Ayush_us_data/"))

In [ ]:
train_labels=[]
train_rest=[]
train_wrist=[]
train_point=[]
train_power=[]
rest=cv2.imread("./Ayush_us_data/avg_frame/rest.jpg",cv2.IMREAD_GRAYSCALE)
point=cv2.imread("./Ayush_us_data/avg_frame/point.jpg",cv2.IMREAD_GRAYSCALE)
wrist=cv2.imread("./Ayush_us_data/avg_frame/wrist.jpg",cv2.IMREAD_GRAYSCALE)
power=cv2.imread("./Ayush_us_data/avg_frame/power.jpg",cv2.IMREAD_GRAYSCALE)


def correlation_img(frame1,frame2):
    m1=np.mean(frame1)
    m2=np.mean(frame2)
    num1=np.sum(np.multiply(np.subtract(frame1,m1),np.subtract(frame2,m2)))
    den1=np.sum(np.multiply(np.emath.sqrt(np.square(np.subtract(frame1,m1))),np.emath.sqrt(np.square(np.subtract(frame2,m2)))))
    return (num1/den1)

In [ ]:
for dir in glob.glob("Ayush_us_data/train/*"):
    label=dir.split("/")[-1]
    print(label)
    for img_name in os.listdir(dir):
        path=os.path.join(dir,img_name)       
        img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        corr_rest=correlation_img(rest,img)
        corr_power=correlation_img(power,img)
        corr_wrist=correlation_img(wrist,img)
        corr_point=correlation_img(point,img)
        train_labels.append(label)
        train_point.append(corr_point)
        train_power.append(corr_power)
        train_wrist.append(corr_wrist)
        train_rest.append(corr_rest)

train_point=np.array(train_point)
train_rest=np.array(train_rest)
train_wrist=np.array(train_wrist)
train_power=np.array(train_power)
train_labels=np.array(train_labels)

point
power
rest
wrist


In [ ]:
test_labels=[]
test_rest=[]
test_wrist=[]
test_point=[]
test_power=[]

In [ ]:
for t_dir in glob.glob("Ayush_us_data/test/*"):
    t_label=t_dir.split("/")[-1]
    print(t_dir)
    for t_img_name in os.listdir(t_dir):
        t_path=os.path.join(t_dir,t_img_name)
        t_img=cv2.imread(t_path, cv2.IMREAD_GRAYSCALE)
        t_corr_rest=correlation_img(rest,t_img)
        t_corr_power=correlation_img(power,t_img)
        t_corr_wrist=correlation_img(wrist,t_img)
        t_corr_point=correlation_img(point,t_img)
        test_labels.append(t_label)
        test_point.append(t_corr_point)
        test_power.append(t_corr_power)
        test_wrist.append(t_corr_wrist)
        test_rest.append(t_corr_rest)


test_point=np.array(test_point)
test_rest=np.array(test_rest)
test_wrist=np.array(test_wrist)
test_power=np.array(test_power)
test_labels=np.array(test_labels)

Ayush_us_data/test/point
Ayush_us_data/test/power
Ayush_us_data/test/rest
Ayush_us_data/test/wrist


In [ ]:
encoder=preprocessing.LabelEncoder()
encoder.fit(test_labels)
encoded_test_labels=encoder.transform(test_labels)
encoder.fit(train_labels)
encoded_train_labels=encoder.transform(train_labels)

In [ ]:
df_train=pandas.DataFrame()
df_train['label']=train_labels
df_train['power']=train_power
df_train['rest']=train_rest
df_train['point']=train_point
df_train['wrist']=train_wrist


In [ ]:
df_test=pandas.DataFrame()
df_test['label']=encoded_test_labels
df_test['power']=test_power
df_test['rest']=test_rest
df_test['point']=test_point
df_test['wrist']=test_wrist


In [ ]:
x_data = df_train.drop(['label'],axis=1)
y_data=df_train['label']
x_test=df_test.drop(['label'],axis=1)
y_test=df_test['label']

In [ ]:
knn_clf=KNeighborsClassifier()
knn_clf.fit(x_data,y_data)
ypred=knn_clf.predict(x_test)

In [ ]:
result=accuracy_score(y_test,ypred)
print(result)

In [ ]:
conf_matrix=confusion_matrix(y_test,ypred)

In [ ]:
sns.heatmap(conf_matrix, annot=True)